In [1]:
import pandas as pd
from dateutil.relativedelta import relativedelta
import numpy as np
import re
import matplotlib.pyplot as plt
from matplotlib import pyplot as plt
from matplotlib.pyplot import figure
import warnings
warnings.filterwarnings("ignore")
import math
import os
from datetime import date, timedelta, datetime
import time
from tqdm import tqdm
import seaborn as sns
from scipy import stats
from matplotlib.ticker import MaxNLocator
from matplotlib.backends.backend_pdf import PdfPages
start_time = time.perf_counter()

In [2]:
price_data = pd.read_csv('stockPriceData-2.csv')
price_data_500 = price_data.groupby('Date', group_keys=False).apply(lambda x: x.sort_values(by='Mcap', ascending=False).head(500))
df = price_data[price_data['Symbol'].isin(price_data_500['Symbol'])]
df['Date'] = pd.to_datetime(df['Date'])
master_date = df.drop_duplicates(subset='Date')[['Date']].reset_index(drop=True)
df.set_index('Date', inplace=True)
df = df.sort_values(by=['Date','Mcap'], ascending=[True,False])
df = df.sort_values(['Symbol', 'Date'])
df['PrevClose'] = df.groupby('Symbol')['Close'].shift(1)
df['returns'] = (df['Close'] - df['PrevClose']) / df['PrevClose']

In [3]:
# Read the data
mom = pd.read_csv('combined_df_nav_rebalanceFrequency_1_topStocks_50_20-06-2025_15.48.23.csv')
mom['Date'] = pd.to_datetime(mom['Date'])
mom = mom.set_index('Date')

In [151]:
# Drop non-numeric columns
numeric_df = mom.select_dtypes(include=['number'])

n = 44
# Compute 22-day rolling returns and vol
rolling_returns = numeric_df / numeric_df.shift(n) - 1
daily_returns = numeric_df.pct_change()
rolling_vol = daily_returns.rolling(n).std()

# Compute returns/vol
returns_over_vol = rolling_returns / rolling_vol

# Compute cross-sectional z-scores per day
cross_sectional_z = returns_over_vol.apply(
    lambda row: (row - row.mean()) / row.std(),
    axis=1
)
# Reset index so Date is a column again
cross_sectional_z = cross_sectional_z.reset_index()

In [152]:
# Apply normalized momentum score logic column-wise
cols_to_process = cross_sectional_z.columns.difference(['Date'])  # all columns except Date
for col in cols_to_process:
    cross_sectional_z[f'{col}'] = np.where(
        cross_sectional_z[col] >= 0,
        1 + cross_sectional_z[col],
        (1 - cross_sectional_z[col]) ** -1
    )

In [153]:
# Assuming cross_sectional_z is your DataFrame
cross_sectional_z['Date'] = pd.to_datetime(cross_sectional_z['Date'])
# Set Date as index for easier calculation
cross_sectional_z = cross_sectional_z.set_index('Date')

# Convert all columns to numeric if they aren't already
numeric_df = cross_sectional_z.apply(pd.to_numeric, errors='coerce')
numeric_df

,PB_df1,PE_df1,Low_Beta_Rank_df2,ROE_df3,ROCE_df3,Asset_Turnover_df3,LowVol_df4,DownsideVol_df4,Operating_profit_Growth_df5,Advertising_Growth_df5,Cash_from_Operation_Growth_df5,Net_Sales_Growth_df5,Adj_PAT_Growth_df5,FCF_Growth_df5,6-1M Momentum_pct_df6,12-1M Momentum_pct_df6,12-1M 6-1M Momentum_pct_df6,3M Momrntum_df7,Short term Reversal Rank_df8
Date,,,,,,,,,,,,,,,,,,,
2007-01-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-13,1.095077,0.524902,0.781204,0.714005,1.979568,1.625187,0.447264,0.610007,2.156269,1.932522,1.255804,2.206861,0.893164,2.774003,0.785369,0.867047,0.346727,0.373510,1.543252
2025-06-16,1.026627,0.608663,0.477289,0.679279,2.137448,1.305206,0.373696,0.472064,2.139566,2.037887,1.207395,2.305761,0.976324,2.602939,1.279490,1.082582,0.414075,0.441778,1.582541
2025-06-17,1.051315,0.666981,0.443389,0.762805,2.439214,1.261733,0.439903,0.529233,2.110305,2.086093,1.296238,2.328115,0.946754,2.416868,0.840939,0.804325,0.352267,0.468796,1.699012


In [136]:
# # Method 1: Keep top 10 values per row, set others to NaN
# def keep_top_10_zscores(df, top_n=5):
#     """
#     Keep only the top N highest z-scores for each date (row).
#     All other values are set to NaN.
#     """
#     result_df = df.copy()
    
#     for index, row in df.iterrows():
#         # Get non-NaN values and their positions
#         non_nan_values = row.dropna()
        
#         if len(non_nan_values) > top_n:
#             # Find the top N highest values
#             top_values = non_nan_values.nlargest(top_n)
#             top_indices = top_values.index
            
#             # Set all values to NaN first
#             result_df.loc[index, :] = np.nan
#             # Then restore only the top N values
#             result_df.loc[index, top_indices] = top_values
#         # If there are <= top_n non-NaN values, keep all of them
    
#     return result_df

# # Apply the function to your DataFrame
# top_10_df = keep_top_10_zscores(numeric_df, top_n=5)

# # Method 2: Alternative approach using rank
# def keep_top_10_zscores_rank(df, top_n=10):
#     """
#     Alternative method using pandas rank function
#     """
#     result_df = df.copy()
    
#     # For each row, rank the values (highest gets rank 1)
#     ranks = df.rank(axis=1, method='min', ascending=False)
    
#     # Keep only values with rank <= top_n
#     mask = ranks <= top_n
#     result_df = result_df.where(mask, np.nan)
    
#     return result_df

# # Alternative implementation
# top_10_df_alt = keep_top_10_zscores_rank(numeric_df, top_n=5)

# # Display sample of results
# print("Original DataFrame (first 5 rows, first 5 columns):")
# print(numeric_df.iloc[:5, :5])
# print("\nTop 10 Z-scores DataFrame (first 5 rows, first 5 columns):")
# print(top_10_df.iloc[:5, :5])

# # Check how many non-NaN values per row (should be <= 10)
# print("\nNumber of non-NaN values per date (should be <= 10):")
# print(top_10_df.count(axis=1).head(10))

In [154]:
# Compute weights per date — option 1: sum to 1
# weights_sum1 = top_10_df.div(top_10_df.sum(axis=1), axis=0)

weights_sum1 = numeric_df.div(numeric_df.sum(axis=1), axis=0)


In [155]:
weights_sum1

,PB_df1,PE_df1,Low_Beta_Rank_df2,ROE_df3,ROCE_df3,Asset_Turnover_df3,LowVol_df4,DownsideVol_df4,Operating_profit_Growth_df5,Advertising_Growth_df5,Cash_from_Operation_Growth_df5,Net_Sales_Growth_df5,Adj_PAT_Growth_df5,FCF_Growth_df5,6-1M Momentum_pct_df6,12-1M Momentum_pct_df6,12-1M 6-1M Momentum_pct_df6,3M Momrntum_df7,Short term Reversal Rank_df8
Date,,,,,,,,,,,,,,,,,,,
2007-01-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-13,0.047795,0.022910,0.034096,0.031163,0.086400,0.070933,0.019521,0.026624,0.094112,0.084346,0.054811,0.096320,0.038983,0.121073,0.034278,0.037843,0.015133,0.016302,0.067356
2025-06-16,0.044346,0.026291,0.020617,0.029342,0.092328,0.056379,0.016142,0.020391,0.092419,0.088027,0.052154,0.099598,0.042173,0.112435,0.055268,0.046763,0.017886,0.019083,0.068359
2025-06-17,0.045820,0.029070,0.019325,0.033246,0.106310,0.054991,0.019173,0.023066,0.091975,0.090920,0.056495,0.101468,0.041263,0.105336,0.036651,0.035056,0.015353,0.020432,0.074049


In [156]:
df1 = pd.read_csv('value.csv', parse_dates=True)
df2 = pd.read_csv('LowBeta.csv', parse_dates=True)
df3 = pd.read_csv('New Quality.csv', parse_dates=True)
df4 = pd.read_csv('lv+dv.csv', parse_dates=True)
df5 = pd.read_csv('New Growth.csv', parse_dates=True)
df6 = pd.read_csv('Mom1.csv', parse_dates=True)
df7 = pd.read_csv('Mom2.csv', parse_dates=True)
df7['3M Momrntum'] = df7.groupby('Date')['3M Momrntum'].rank(pct=True)
df8 = pd.read_csv('ShortTermReversal.csv', parse_dates=True)

from functools import reduce

# List of all DataFrames
dfs = [df1, df2, df3, df4, df5, df6, df7, df8]

# Add suffixes to identify columns from each DF (except 'Date' and 'Symbol')
for i, df in enumerate(dfs, 1):
    df.columns = ['Date', 'Symbol'] + [f"{col}_df{i}" for col in df.columns if col not in ['Date', 'Symbol']]

# Merge all using reduce
merged_df = reduce(lambda left, right: pd.merge(left, right, on=['Date', 'Symbol'], how='outer'), dfs)

# Optional: sort and reset index
merged_df = merged_df.sort_values(['Date', 'Symbol']).reset_index(drop=True)

merged_df = merged_df[merged_df['Date'] >= '2007-01-19']

In [157]:
# Set Date as index for both
merged_df['Date'] = pd.to_datetime(merged_df['Date'])

# Merge weights_df onto merged_df on Date to repeat the weights for each symbol
merged_with_weights = merged_df.merge(weights_sum1, on='Date', suffixes=('', '_wgt'))

# Get the list of signal columns (the ones to multiply)
signal_cols = [c for c in merged_df.columns if c not in ['Date', 'Symbol']]

# Multiply each column by its weight column
for col in signal_cols:
    merged_with_weights[col] = merged_with_weights[col] * merged_with_weights[col + '_wgt']

# Drop weight columns if not needed
merged_weighted_df = merged_with_weights.drop(columns=[c + '_wgt' for c in signal_cols])

merged_weighted_df

,Date,Symbol,PB_df1,PE_df1,Low_Beta_Rank_df2,ROE_df3,ROCE_df3,Asset_Turnover_df3,LowVol_df4,DownsideVol_df4,...,Advertising_Growth_df5,Cash_from_Operation_Growth_df5,Net_Sales_Growth_df5,Adj_PAT_Growth_df5,FCF_Growth_df5,6-1M Momentum_pct_df6,12-1M Momentum_pct_df6,12-1M 6-1M Momentum_pct_df6,3M Momrntum_df7,Short term Reversal Rank_df8
0,2007-01-19,3IINFOLTD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2007-01-19,3MINDIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2007-01-19,63MOONS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2007-01-19,ABAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2007-01-19,ABB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2368929,2025-06-13,ZENSARTECH,0.023994,0.015321,NaN,0.020726,0.065672,0.051199,0.008048,0.013509,...,0.032178,0.016693,0.022298,0.034270,0.058359,0.028962,0.030787,0.012645,0.008375,0.046756
2368930,2025-06-13,ZENTEC,0.013149,0.006628,NaN,NaN,NaN,NaN,0.000660,0.000338,...,NaN,NaN,NaN,NaN,NaN,0.021409,0.033962,0.013132,0.015027,0.060038
2368931,2025-06-13,ZFCVINDIA,0.019195,0.009270,NaN,0.013694,0.051306,0.054044,0.010524,0.009738,...,NaN,0.009069,0.045516,0.021995,0.033680,0.021642,0.019467,0.008067,0.008858,0.021820
2368932,2025-06-13,ZYDUSLIFE,0.029560,0.019980,NaN,0.020800,0.059926,0.061333,0.016673,0.019138,...,0.033641,0.030100,0.053332,0.033386,0.087974,0.016268,0.013277,0.005489,0.006100,0.047705


In [158]:
merged_weighted_df['Total_Score'] = merged_weighted_df[cols_to_process].sum(axis=1)

# Keep Date, Symbol, Total_Score
scores_df = merged_weighted_df[['Date', 'Symbol', 'Total_Score']]
print(scores_df.head())

        Date     Symbol  Total_Score
0 2007-01-19  3IINFOLTD          0.0
1 2007-01-19    3MINDIA          0.0
2 2007-01-19    63MOONS          0.0
3 2007-01-19       ABAN          0.0
4 2007-01-19        ABB          0.0


In [142]:
merged_weighted_df[merged_weighted_df['Symbol'] == 'ZYDUSWELL'].tail(1)

,Date,Symbol,PB_df1,PE_df1,Low_Beta_Rank_df2,ROE_df3,ROCE_df3,Asset_Turnover_df3,LowVol_df4,DownsideVol_df4,...,Cash_from_Operation_Growth_df5,Net_Sales_Growth_df5,Adj_PAT_Growth_df5,FCF_Growth_df5,6-1M Momentum_pct_df6,12-1M Momentum_pct_df6,12-1M 6-1M Momentum_pct_df6,3M Momrntum_df7,Short term Reversal Rank_df8,Total_Score
2368933,2025-06-13,ZYDUSWELL,0.094052,0.002483,NaN,0.004473,0.002861,0.005438,0.05483,0.090803,...,0.077407,0.016855,0.006397,0.049892,0.013172,0.01369,0.011712,0.008878,0.024541,0.504688


In [159]:
# Filter dates where at least one non-zero score exists
nonzero_dates = scores_df.loc[scores_df['Total_Score'] != 0, 'Date'].unique()

filtered_df = scores_df[scores_df['Date'].isin(nonzero_dates)]

# For each date, pick top 25 stocks by Total_Score
top25_df = (
    filtered_df.sort_values(['Date', 'Total_Score'], ascending=[True, False])
    .groupby('Date')
    .head(25)
    .reset_index(drop=True)
)

print(top25_df)

             Date      Symbol  Total_Score
0      2007-03-28     63MOONS     0.785576
1      2007-03-28        MFSL     0.778016
2      2007-03-28   GEOJITFSL     0.775626
3      2007-03-28  ASHAPURMIN     0.740130
4      2007-03-28     HAVELLS     0.726214
...           ...         ...          ...
112345 2025-06-13      BIKAJI     0.642694
112346 2025-06-13         BEL     0.642034
112347 2025-06-13   APLAPOLLO     0.640785
112348 2025-06-13   HINDPETRO     0.640030
112349 2025-06-13  BERGEPAINT     0.636856

[112350 rows x 3 columns]


In [160]:
price = pd.read_csv('stockPriceData-2.csv')
price = price.sort_values(by=['Date','Mcap'], ascending=[True,False])
price['Date'] = pd.to_datetime(price['Date'])
price = price[price['Date'] >= '2007-01-02']

master_date = price.drop_duplicates(subset='Date')[['Date']].reset_index(drop=True)
master_date = master_date.sort_values(by='Date')
master_date = master_date[master_date['Date'] >= '2006-01-01']
master_date

,Date
0,2007-01-02
1,2007-01-03
2,2007-01-04
3,2007-01-05
4,2007-01-08
...,...
4546,2025-06-09
4547,2025-06-10
4548,2025-06-11
4549,2025-06-12


In [161]:
portfolio = top25_df.groupby("Date")["Symbol"].apply(list).reset_index()
portfolio['Date'] = pd.to_datetime(portfolio['Date'])

# Expand portfolio by date
merged = pd.merge_asof(master_date, portfolio, on="Date")

merged = merged.explode("Symbol").dropna(subset=["Symbol"])
merged = merged[merged["Date"]>= '2007-02-23'].reset_index(drop=True)
# Define the start date (it should start from 2006-06-19)
start_date = pd.to_datetime('2007-02-23')

# Filter the dates starting from 2006-06-19 (this ensures the start date is included)
filtered_dates = master_date[master_date['Date'] >= start_date]

# Ensure the start date is included
start_date_row = filtered_dates[filtered_dates['Date'] == start_date]

# Select every 50th date from the filtered dates (starting from the filtered list after the start date)
selected_dates = filtered_dates.iloc[::44]

# Combine the start date with the selected dates
selected_dates = pd.concat([start_date_row, selected_dates])

# Drop any NaN values and sort by date to ensure it's in the correct order
selected_dates = selected_dates.dropna().sort_values(by='Date')

combined_dates_portfolio = merged[merged['Date'].isin(selected_dates['Date'])]
combined_dates_portfolio

,Date,Symbol
550,2007-04-30,GEOJITFSL
551,2007-04-30,AMBUJACEM
552,2007-04-30,ASHAPURMIN
553,2007-04-30,DALMIASUG
554,2007-04-30,JINDALSAW
...,...,...
111670,2025-05-07,ANANDRATHI
111671,2025-05-07,PIDILITIND
111672,2025-05-07,ALKEM
111673,2025-05-07,INDIGO


In [ ]:
combined_dates_portfolio.to_csv('.csv',index=False)

In [163]:
combined_dates_portfolio

,Date,Symbol
550,2007-04-30,GEOJITFSL
551,2007-04-30,AMBUJACEM
552,2007-04-30,ASHAPURMIN
553,2007-04-30,DALMIASUG
554,2007-04-30,JINDALSAW
...,...,...
111670,2025-05-07,ANANDRATHI
111671,2025-05-07,PIDILITIND
111672,2025-05-07,ALKEM
111673,2025-05-07,INDIGO
